### Takes my smorf dataset and uses NeuroSEED's code to store it in the same way

In [11]:
import sys
import re
import numpy as np
sys.path.insert(0,'../')

import pandas as pd
import argparse

from edit_distance.task.dataset_generator_genomic import EditDistanceGenomicDatasetGenerator

In [12]:
df = pd.read_csv("../datasets/dataset_FINAL.tsv", sep='\t')

df_subset=df.dropna().sample(100, random_state=42)

if not os.path.exists('processed_data'):
    os.makedirs('processed_data')
    
df_subset.to_csv('processed_data/smorf_subset.csv')
df_subset.head()

,set,clust,name,origin,y,upstream,downstream,smorf
60287,train,smorfam02935,SRS144135|31130|129|275|-1,real,positive,GATTAATAAACAAAAAAGAAATAATTCGTTGACTAAAATGTAAAAA...,AATGATTCTTTCTGTAATAAAAAAGTAAAAATGTAATTAAAAAGTA...,ATGTTAAATGACGAAATATGTAAATTAAGAGATAGATTAAACGAAA...
38271,train,smorfam00398,SRS049896|100_4224.208254|61|198|-1,real,positive,AGGACATTTATCGCTTTCAGACCTATCTGGAAGATCACAGCACCAC...,AAACACCCTGTCGGTGCTGAAAATTGCACCGGGACAGCATCCGCAG...,ATGCTGAACTTTTTTATCGGTATCGCATTTTTTGAGGCTGGTGCCT...
447517,dev,195481,SRS146830|1340|13074|13202|1,fake,negative,CGAATAGAATACAATTTTATTATAATGGTAAAGCTGAAAAGCCGAG...,AATGCCTTTAGACCCCAAGTTTTGTACTGACATGTAGATTTTATTA...,ATGACGCTCTTATTTTTCAATGTCCTCCGTATTGCAAGAATTTATT...
255578,train,smorfam01930,SRS015578|100_386.194237|8|127|-1,fake,negative,AATTTTATTAAATGAATTTGTAAATGTGTTTGTTTTATGTCAAATA...,CTGGATG,ATGGAAATAGAGTTGAATTGTTGGTCCGGTTATAGTTTAAATTTAA...
392941,train,70008,SRS055966|100_160.161|2113|2259|1,fake,negative,TAGGGATCACCTCCGGAGGGCAGGTGCGGCTGGAATCTTCTAGATA...,TAGCACTTATGATCCGATCCAAACAGTGAGAATTCACAAGAAGACT...,GTGGATTCTGCGATGTCGATGGAGAGAGCCGGGTCGCCCGGGGGGA...


In [45]:
# save dataset as string

file_out='../datasets/strings.txt'

no_weird_chars=df['smorf'].str.contains(r'^[ACTG]+$', na=False)

print(len(df[no_weird_chars].smorf.values))

with open(file_out, 'w') as f_out:
    f_out.writelines("%s\n" % l for l in df[no_weird_chars].smorf.values)



452020


In [47]:
# create string to be read by NeuroSEED-borrowed code
parser = argparse.ArgumentParser()
parser.add_argument('--out', type=str, default="../datasets/string_subset.pkl", help='Output data path')
parser.add_argument('--train_size', type=int, default=7000, help='Training sequences')
parser.add_argument('--val_size', type=int, default=700, help='Validation sequences')
parser.add_argument('--test_size', type=int, default=1400, help='Test sequences')
parser.add_argument('--source_sequences', type=str, default=file_out, help='Sequences data path')
args, unknown = parser.parse_known_args()



# load and divide sequences
with open(args.source_sequences, 'rb') as f:
    L = f.readlines()
L = [l[:-1].decode('UTF-8') for l in L]

strings = {
    'train': L[:args.train_size],
    'val': L[args.train_size:args.train_size + args.val_size],
    'test': L[args.train_size + args.val_size:args.train_size + args.val_size + args.test_size]
}

data = EditDistanceGenomicDatasetGenerator(strings=strings)
data.save_as_pickle(args.out)




Generating train


Edit distance 7000x7000:  55%|█████▍    | 3848/7000 [02:24<01:53, 27.88it/s]